# Variational Quantum Eigensolver for 2-qubit system

This is an attempt at solving task 4 of the screening tasks i.e. finding the lowest eigenvalue for the given matrix using VQE like circuits.

## Introduction
Variational Quantum Eigensolver is an algorithm that helps us find an upper bound on the lowest eigen value of a given Hamiltonian. This notebook will not go into much depth on the theoretical aspects of the algorithm such as the variational principle and what a Hamiltonian. This article<sup>[[1]]</sup> by Michał Stęchły and the original paper on VQE<sup>[[2]]</sup> do a great job of explaining them. Here, the focus will be on the implementation of the algorithm.  
<br>

<div align="center">
    <img alt="Variational Quantum Algorithm" src="./VQE_algorithm.png"><br>
    Variational Quantum Algorithm<sup>[2]</sup>
</div>

<br><br>
The steps involved in the algorithm can be summarized as follows:  

1. Design a parameterized circuit (ansatz) for trial state preparation. Design the quantum modules based on the tensor products of Pauli matrices obtained by decomposing the Hamiltonian.  

2. Initialize the parameters of the circuit and prepare a trial state.  

3. Pass the trial state to the quantum modules to perform measurements.  

4. Calculate the expectation value $\left\langle H_{1} \right\rangle$ to $\left\langle H_{n} \right\rangle$ based on the counts of measurements obtained from quantum modules.  

5. Add all the expectation values using classical adder to obtain expectation value $\left\langle H \right\rangle$.  

6. Obtain new input parameters based on the expectation value using a classical optimizer.  

7. Repeat steps 2 to 7 until the expectation value is optimised. This expectation value is the upper bound on the lowest eigenvalue of the given Hamiltonian.


[1]: (https://www.mustythoughts.com/variational-quantum-eigensolver-explained)
[2]: (https://arxiv.org/abs/1304.3061)


Before we proceed with the code, let's make sure that all the required dependencies are installed on your system. It is recommended that these dependencies be installed in a virtual environment so that the global ones are not messed up. 


In [1]:
!pip install -U -r requirements.txt

from IPython.display import clear_output
clear_output()

In [2]:
''' Importing the required modules and defining global constants '''
import numpy as np
pi = np.pi
from qiskit import *
from scipy import optimize
import pandas as pd
from matplotlib.pyplot import savefig

## Decomposing given Hamiltonian into Pauli products

The first step in the algorithm requires us to decompose the given Hamiltonian matrix into a linear combination of Pauli matrices and tensor products of Pauli matrices (henceforth called "Pauli product"). This is done as a quantum computer can efficiently evaluate the expectation value of Pauli products<sup>[[2]]</sup>.

After digging around the internet,  these resources<sup>[[3]][[4]][[5]][[8]]</sup> were found that leverage the properties of Pauli matrices, Hermitian matrices and the Hilbert-Schmidt inner product to calculate the coefficients of the decomposition. If we represent the decomposed 4 x 4 Hamiltonian matrix H as

$$H = \ \sum_{i,j = I,X,Y,Z}^{}{a_{\text{ij}}\left( \sigma_{i} \otimes \sigma_{j} \right)}$$

then we can use the relation

$$a_{\text{ij}} = \frac{1}{4}tr(\left( \sigma_{i} \otimes \sigma_{j} \right)H)\ where\ i,\ j = I,\ X,\ Y,\ Z$$

to calculate the coefficient of each Pauli product.

[2]: https://arxiv.org/abs/1304.3061
[3]: https://quantumcomputing.stackexchange.com/questions/11899/example-of-hamiltonian-decomposition-into-pauli-matrices
[4]: https://quantumcomputing.stackexchange.com/questions/8725/can-arbitrary-matrices-be-decomposed-using-the-pauli-basis
[5]: https://quantumcomputing.stackexchange.com/questions/6882/decomposition-of-a-matrix-in-the-pauli-basis
[8]: https://michaelgoerz.net/notes/decomposing-two-qubit-hamiltonians-into-pauli-matrices.html


In [3]:
def decompose_matrix(matrix):
    """ 
    This function uses the formula describe above to calculate the coefficient of each Pauli product. 
  
    It essentially decomposes the 4 x 4 hamiltonian matrix into a linear combination of Pauli products.  
  
    Parameters: 
    matrix (np.array): A 4 x 4 hamiltonian matrix
  
    Returns: 
    dict: Dictionary of coefficients of each Pauli product
    """

    pauli_I = np.array([[1,  0], 
                        [0,  1]], dtype=complex)
    pauli_X = np.array([[0,  1], 
                        [1,  0]], dtype=complex)
    pauli_Y = np.array([[0, -1j], 
                        [1j,  0]], dtype=complex)
    pauli_Z = np.array([[1,  0], 
                        [0, -1]], dtype=complex) 
    pauli_matrices = [["I", pauli_I], ["X", pauli_X], ["Y", pauli_Y], ["Z", pauli_Z]]

    coefficient_dict = {}

    for pauli_matrix_1 in pauli_matrices:
        for pauli_matrix_2 in pauli_matrices:
            tensor_product = np.kron(pauli_matrix_1[1], pauli_matrix_2[1])
            coefficient_dict[f"{pauli_matrix_1[0]}{pauli_matrix_2[0]}"] = 0.25 * np.trace(np.matmul(tensor_product, given_matrix))

    return coefficient_dict

In [4]:
given_matrix = np.array([[1,  0,  0,  0], 
                         [0,  0, -1,  0],
                         [0, -1,  0,  0],
                         [0,  0,  0,  1]], dtype=np.float64)
print("Coefficient of each tensor product in the decomposition of given matrix: \n\n", decompose_matrix(given_matrix))

Coefficient of each tensor product in the decomposition of given matrix: 

 {'II': (0.5+0j), 'IX': 0j, 'IY': 0j, 'IZ': 0j, 'XI': 0j, 'XX': (-0.5+0j), 'XY': 0j, 'XZ': 0j, 'YI': 0j, 'YX': 0j, 'YY': (-0.5+0j), 'YZ': 0j, 'ZI': 0j, 'ZX': 0j, 'ZY': 0j, 'ZZ': (0.5+0j)}


It is seen from the above output that the given matrix is decomposed into a linear combination of $\sigma_{I} \otimes \sigma_{I}$, $\sigma_{X} \otimes \sigma_{X}$, $\sigma_{Y} \otimes \sigma_{Y}$ and $\sigma_{Z} \otimes \sigma_{Z}$ i.e.

$$H = \frac{1}{2}\left( \sigma_{I} \otimes \sigma_{I} \right) - \frac{1}{2}\ \left( \sigma_{X} \otimes \sigma_{X} \right) - \frac{1}{2}{(\sigma}_{Y} \otimes \sigma_{Y}) + \frac{1}{2}\left( \sigma_{Z} \otimes \sigma_{Z} \right)$$

## Creating Trial States using Parametrized Ansatz

The next step is to design the circuit for trial state preparation. The goal is to create a state that is exactly the eigen state or very close to it. We could do this by iterating over all possible states in the Hilbert Space until we find the correct one but this would be computationally intensive. We need a better way to do so. 

An ansatz, in general sense, is an educated guess or an additional assumption made to help solve a problem, and which is later verified to be part of the solution by its results<sup>[[6]]</sup>. In this case, an ansatz is a set of parametrized gates that gives us access to a portion of the Hilbert space. Since it is parameterized, the parameters can be varied iteratively to access the set of states represented by it. Choosing a good ansatz is key as it should represent a sufficient part of the Hilbert space and be shallow (to be less computationally intensive) and should not have too many parameters (optimizing it would become difficult)<sup>[[1]]</sup>. 

After experimenting with different ansatze, it was found that the ansatz in the hint i.e. ${RX(\theta)}_{1}{(CX)H}_{1}$ was the best ansatz for the given hamiltonian as it contains the eigenvector, corresponding to the lowest eigenvalue, as a state in its subspace. 

[6]: https://en.wikipedia.org/wiki/Ansatz
[1]: https://www.mustythoughts.com/variational-quantum-eigensolver-explained


In [5]:
def create_trial_state_circuit(parameters):
    """ 
    Creates a parameterized circuit (ansatz) that prepares the trial state based 
    on the parameters received as input.
  
    Parameters: 
    parameters (np.array): List of angles that act as parameters for the circuit
  
    Returns: 
    QuantumCircuit(2, 2): Quantum circuit that prepares the trial state
    """
    trial_state_circuit = QuantumCircuit(2, 2)

    trial_state_circuit.h(0)
    trial_state_circuit.cx(0, 1)
    trial_state_circuit.rx(parameters[0], 0)

    trial_state_circuit.barrier()
    # trial_state_circuit.draw(output='mpl').savefig('./circuits/AnsatzHint.png') # This statement was used during the experimentation phase to store the circuits
    return trial_state_circuit

## Creating quantum modules to perform measurements

It is impossible to know the exact state of a qubit as any external interaction collapses the qubit into one of the basis states. Sp, to get an approximate idea of what the trial state would have been, the same circuit is repeatedly prepared and measurements are performed measurements to get the counts of each output state. These can be used to calculate the probability of each output state and which can in turn be used to calculate the expectation value.

The problem that arises here is that separate circuits (i.e. quantum modules) are needed for each Pauli Product. The reason being that to calculate the expectation value for $\sigma_{X}$ and $\sigma_{y}$, measurements have to be performed and counts have to be obtained in the X and Y basis respectively. Since we cannot directly measure in an arbitrary basis, transformations have to be performed on the trial state to convert it into Z basis (i.e. the $\left| 0 \right\rangle\ and\ |1\rangle$ basis). For $\sigma_{X}$, a Hadamard (or $H$) gate is applied and for $\sigma_{y}$, $S^{\dagger}$ followed by $H$ gate are applied for the transformation.

$$S^{\dagger} = \ \begin{bmatrix} 1 & 0 \\ 0 & - i \\ \end{bmatrix}\ \ \ \ \ \ \ \ \ \ H = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & - 1 \\ \end{bmatrix}$$

For $\sigma_{Z}$, there is no need for a transformation as its expectation value requires counts in the Z basis. For $\sigma_{I}$, there is no need to create a circuit as its expectation value is always 1. 

For a 2-qubit system, the expectation value for tensor products has to be calculated. This can be done by doing these transformations on individual qubits according to the Pauli matrix and then doing a measurement. E.g. For calculating the expectation value of the $\sigma_{X} \otimes \sigma_{Y}$, a $H$ gate is applied on the first qubit and a $S^{\dagger}$ followed by $H$ gate are applied on the second qubit and measurements are performed. 

In [6]:
def quantum_module_simulator(trial_state_circuit, pauli_product, number_of_shots):
    """ 
    This is a generalized function that adds transformations and performs measurements on the trial
    state based on the pauli matrices. The measurements are performed repeatedly to obtain counts of 
    each output state. The measurements are performed on an ideal simulator.
  
    Parameters: 
    trial_state_circuit (QuantumCircuit): Circuit that prepares the trial state
    pauli_product (str): String representation of tensor product of pauli matrices
    number_of_shots (int): Number of times measurements should be performed on the trial state
    
    Returns: 
    counts (dict): Dictionary of counts of each output state
    """

    measurement_circuit = trial_state_circuit.copy()

    qubit = 0
    cbit = 0

    for pauli_matrix in pauli_product:
        if pauli_matrix == 'X':
            measurement_circuit.h(qubit)
        elif pauli_matrix == 'Y':
            measurement_circuit.sdg(qubit)
            measurement_circuit.h(qubit)
        elif pauli_matrix != 'I' and pauli_matrix != 'Z':
            raise ValueError("Pauli product should consist only of I, X, Y or Z matrices")
        measurement_circuit.measure(qubit, cbit)
        qubit += 1
        cbit += 1

    backend = Aer.get_backend('qasm_simulator')
    job = execute(measurement_circuit, backend, shots = number_of_shots)
    result = job.result()
    counts = result.get_counts()

    return counts

## Calculating expectation values on the basis of counts

The quantum modules give us a set of counts for each basis state i.e.$\ \left| 00 \right\rangle,\ \left| 01 \right\rangle,$$\ \left| 10 \right\rangle\ and\ |11\rangle$. The probability of each state is calculated by dividing the count of each state by the total number of counts.

The expectation value is the sum of product of probabilities of each state and their associated eigen value. As we know, the eigen value of state $\left| 0 \right\rangle,\left| + \right\rangle$, and $\left| i \right\rangle$ is $+ 1$ and of state $\left| 1 \right\rangle,\left| - \right\rangle$, and $\left| - i \right\rangle$ is $- 1$. The eigen value of tensor products of these states would then be a product of their individual eigen values. Since every state has been transformed into the Z basis, the eigenvalues of only 4 states need to tbe considered. It comes out to be $+ 1$ for $|00\rangle$ and $|11\rangle$ and $- 1\ $for $|01\rangle$ and $|10\rangle$.

The formula used for the calculation depends on the Pauli product used but can be generalised to three cases:

1.  When the Pauli product is $\sigma_{I} \otimes \sigma_{X}$, $\sigma_{I} \otimes \sigma_{Y}$ or $\sigma_{I} \otimes \sigma_{Z}$:

The expectation value in these cases depends only on the second qubit as the first Pauli matrix is $I$. There is no need to create a totally new circuit in these cases and the probabilities obtained from the quantum modules can be used. The eigenvalue of the state is considered as the eigenvalue of state of qubit 2 i.e. states having 1 on the second qubit have $- 1$ and for the states having on the first qubit have $+ 1$. Hence, the expectation value for this case is:

$$\left\langle \sigma_{I} \otimes \sigma_{i} \right\rangle\  = P_{00}\left( + 1 \right) + P_{01}\left( - 1 \right) + P_{10}\left( + 1 \right) + P_{11}( - 1)\ where\ i = X,\ Y\ and\ Z\ $$

2.  When the Pauli product is $\sigma_{X} \otimes \sigma_{I}$, $\sigma_{Y} \otimes \sigma_{I}$ or $\sigma_{Z} \otimes \sigma_{I}$:

Similar to the above case, the expectation value would depend only on the first qubit as the second Pauli matrix is $I$. Similar to the above case, the eigenvalue for states having 1 on the first qubit is considered as $- 1$ and for the states having 0 on the first qubit is considered as $+ 1$. Therefore, the expectation value for this case is:

$$\left\langle \sigma_{i} \otimes \sigma_{I} \right\rangle\  = P_{00}\left( + 1 \right) + P_{01}\left( + 1 \right) + P_{10}\left( - 1 \right) + P_{11}( - 1)\ where\ i = X,\ Y\ and\ Z\ $$

3.  When the Pauli product is of the form $\sigma_{i} \otimes \sigma_{j}$ where $i,\ j = X,\ Y\ and\ Z$:

In this case, the eigen value of the entire state is considered. The eigen value of the 4 states was defined initially and is used here. Therefore, the expectation value for this case is:

$$\left\langle \sigma_{i} \otimes \sigma_{j} \right\rangle\  = P_{00}\left( + 1 \right) + P_{01}\left( - 1 \right) + P_{10}\left( - 1 \right) + P_{11}\left( + 1 \right)\ where\ i,j = X,\ Y,\ Z$$

This tutorial<sup>[[7]](https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/variational_quantum_eigensolver.ipynb)</sup> by David Khachatryan goes into much depth on the significance of adding extra gates before measurement and calculating expectation values and can be referred for further details.

In [7]:
def calculate_expectation_value(counts, pauli_product):
    """ 
    Calculates the expectation value of the Pauli product based on the counts of each state and the formula defined above
  
    Parameters: 
    counts (dict): Dictionary of counts of each output state
    pauli_product (str): String representation of tensor product of pauli matrices
    
    Returns: 
    expectation_value (int): Expectation value of the Pauli product based on the counts 
    """

    if pauli_product == 'II':
        return 1
    
    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_count = counts['00'] + counts['01'] + counts['10'] + counts['11']


    # The formulae vary slightly from above as Qiskit has reverse ordering of qubits (little-endian)
    # i.e. qubit 1 is qubit 2 and qubit 2 is qubit 1

    if pauli_product == 'IX' or pauli_product == 'IY' or pauli_product == 'IZ': 
        expectation_value = (counts['00'] + counts['01'] - counts['10'] - counts['11']) / total_count
    elif pauli_product == 'XI' or pauli_product == 'YI' or pauli_product == 'ZI':
        expectation_value = (counts['00'] - counts['01'] + counts['10'] - counts['11']) / total_count
    else:
        expectation_value = (counts['00'] - counts['01'] - counts['10'] + counts['11']) / total_count

    return expectation_value

## Combining everything to calculate the expectation value

All the parts needed to calculate the expectation value of the given Hamiltonian are defined above. We use the coefficients of pauli products (obtained by decomposing the matrix) and set of parameters for the ansatz to call the respective functions and calculate expectation values for each Pauli product. These expectation values are multiplied with their respective coefficients and added to give the expectation value of the given Hamiltonian with respect to the current trial state.

In [8]:
def calculate_expectation_value_of_hamiltonian(parameters, coefficient_dict):
    """ 
    Calculates the expectation value of the hamiltonian using the parameters for trial state circuit
    and coefficients of pauli products
  
    Parameters: 
    parameters (np.array): List of angles that act as parameters for the trial state circuit 
    coefficient_dict (dict): Coeffiecients of pauli products obtained by decomposing the hamiltonian
    
    Returns: 
    expectation_value_of_hamiltonian (float): Expectation value of the hamiltonian
    """

    trial_state_circuit = create_trial_state_circuit(parameters)

    expectation_value = 0

    for pauli_product in coefficient_dict:
        if abs(coefficient_dict[pauli_product]) > 0:
            counts = quantum_module_simulator(trial_state_circuit, pauli_product, 8192)
            expectation_value += np.real(coefficient_dict[pauli_product]) * calculate_expectation_value(counts, pauli_product)
    
    return expectation_value            

## Optimizing the expectation value of the Hamiltonian

Using the above methods, the expectation value with respect to the current trial state is obtained. It needs to be optimised as it may not be the lowest upper bound on the eigenvalue. Classical optimisation methods such as gradient descent or Nelder-Mead Simplex method can be used to optimize the above function and obtain the lowest possible value. In this notebook, Powell's method is used for optimization. I have also shown the lowest eigen value that is calculated using classical methods for comparison.


In [9]:
given_matrix = np.array([[1,  0,  0,  0], 
                         [0,  0, -1,  0],
                         [0, -1,  0,  0],
                         [0,  0,  0,  1]], dtype=np.float64) 
                         
coefficient_dict = decompose_matrix(given_matrix)

theta1, theta2, theta3, theta4 = 0, 0, 0, 0
parameters = [theta1, theta2, theta3, theta4]
tolerance = 1e-5

''' Running classical algorithm on the given matrix to find exact value '''
eigenvalues = np.linalg.eigvals(given_matrix)
lowest_eigenvalue = np.min(eigenvalues)

print("Classically calculated eigenvalues of given hamiltonian are: ", eigenvalues)
print("Lowest eigenvalue calculated using classical methods: ", lowest_eigenvalue)

result = optimize.minimize(fun=calculate_expectation_value_of_hamiltonian, x0=parameters, 
                           args=(coefficient_dict), method='Powell', tol=tolerance)

print("Upper bound on lowest eigenvalue calculated using VQE: ", result.fun)

Classically calculated eigenvalues of given hamiltonian are:  [ 1. -1.  1.  1.]
Lowest eigenvalue calculated using classical methods:  -1.0
Upper bound on lowest eigenvalue calculated using VQE:  -1.0


As it is seen above, the VQE algorithm is able to return the exact lowest eigenvalue for the given Hamiltonian for the selected ansatz (the classical method may give a small error which can be neglected). 

## Observations and Results

### Selection of Ansatz
As I mentioned earlier, I tried several different ansatze and kept a track of the expectation value of the Hamiltonian obtained using the ansatze. My initial approach was to try several ansatze randomly and observe the effect they have on the expectation value. Some of my initial guesses were based on lectures 25 to 27 by IBM in the course Introduction to Quantum Computing and Quantum Hardware<sup>[[9]]</sup>.

I obtained the expectation value for each ansatz thrice to minimize variation due to random seed of the noiseless simulator. I present the output for each ansatz in the form of a table. It is stored in a file called experimental_data.csv. Due to lack of space I do not print the description of each ansatz below. Also, it would be difficult to show the plots of the circuit along with the expectation values in 1 table, so I have stored the plots of each circuit with their previous parameters in the circuits folder of the repository and can be viewed from there. In all cases, the number of shots for each circuit is fixed at 8192.

[9]: https://qiskit.org/learn/intro-qc-qh/
[10]: https://arxiv.org/abs/1905.10876



In [10]:
experimental_data = pd.read_csv("experimental_data.csv")
experimental_data_without_description = experimental_data.loc[:, 'ID':]

experimental_data_without_description.head(9)

,ID,Number of gates,Number of parameters,Expectation Value 1,Expectation Value 2,Expectation Value 3
0,Ansatz1,4,2,-0.002686,-0.011108,-0.006226
1,Ansatz2,4,4,-0.002563,0.000244,-0.009277
2,Ansatz3,4,2,-0.014160,-0.013062,-0.014771
3,Ansatz4,4,4,-0.010742,-0.015503,0.000122
4,Ansatz5,9,2,-0.261841,-0.269043,-0.266235
5,Ansatz6,14,2,-0.061035,-0.062500,-0.057007
6,Ansatz7,9,4,-0.505371,-0.511108,-0.499512
7,Ansatz8,14,4,-0.947998,-0.509644,-0.945435
8,AnsatzHint,3,1,-1.000000,-1.000000,-1.000000


The following observations are made based on the above experimental data:

1. In case of Ansatz1, Ansatz2, Ansatz3 and Ansatz4, the expectation values that are obtained are 10<sup>4</sup> to 10<sup>2</sup> orders of magnitude away from the lowest eigenvalue. Hence, they are not good ansatze. The reason behind this is that the all these ansatz are neither expressible nor represent a large number of states. So, even after optimization the circuit would not give an eigen value of -1.

Note: Entanglement was introduced in all circuits from Ansatz5, which increases the expressibility of the ansatz

2. In case of Ansatz 5, the number of gates have been increased and entanglement has been introduced. So, the circuit is expressible. But, the subspace represented by the ansatz still does not contain the exact state represented by the required eigen vector. Hence, we are still an order of magnitude away from the solution.

3. In case of Ansatz 6, the number of gates have been further increased but this worsens the expectation value. This might be due the fact that we have only 2 parameters and adding more gates makes the task of the optimiser difficult without giving a better solution.

4. In case of Ansatz 7, we add more parameters to the circuit while keeping the number of gates constant. This increases the expressibility further and now we get a much better upper bound. The drawback is that the circuit takes longer to execute and more time is taken per iteration.

5. In case of Ansatz 8, we increase the gates again but this time we have 4 parameters instead of 2. This is the best I could do with my circuit and was the closest expectation value I got. The drawback here was same as Ansatz 7 and I had to look for alternatives.

6. I had a look at the ansatz given in the hint and applied it here. It had only 3 gates and 1 parameter so I was suspicious. But surprisingly, it gave the exact eigenvalue of -1 each time I executed it without taking long to execute. Later, I deduced that the subspace represented by AnsatzHint contains the exact eigenvector that we need and varying the parameter would always lead us to the exact solution. Since, the circuit is small, it is not expressible but it works in this case.

After analysing all these observations, one can see that we can get a good (or exact) upper bound on the lowest eigen value if we know the subspace in which the corresponding eigenvector lies. We can design very shallow circuits which sweep that subspace to get the eigenvalue. This approach cannot be generalised as in large and random hamiltonians we would not know the subspace in which the eigenvectors lie and we need expressible circuits. In such cases, we need generalised circuits like Ansatz8 which would take more time but give us a good upper bound.

### Introducing noise into the circuit

I ran the circuit having AnsatzHint which gave an exact eigen value of -1 on the noiseless simulator on a noisy IBM quantum computer (ibmq_15_melbourne). I observed that even this shallow circuit could not give an exact value on the noisy device and gave an upper bound of -0.9023. This effect of noise is drastic and we need better quantum devices to get exact eigen values in the future.

## Conclusion

I want to conclude by saying that it was a great experience working on NISQ algorithm and learning it in depth. The main outcomes for me were that choosing an ansatz for such circuits is an art and it would take a lot of time to develop an intuition in it. I also learnt that there are several factors that can affect the outcome of a variational circuit such as cost of the circuit, expressibility of the circuit, number of parameters, noise of the device, etc. and striking the right balance is important to get a good result. 

## References

1. https://www.mustythoughts.com/variational-quantum-eigensolver-explained
2. https://arxiv.org/abs/1304.3061
3. https://quantumcomputing.stackexchange.com/questions/11899/example-of-hamiltonian-decomposition-into-pauli-matrices
4. https://quantumcomputing.stackexchange.com/questions/8725/can-arbitrary-matrices-be-decomposed-using-the-pauli-basis
5. https://quantumcomputing.stackexchange.com/questions/6882/decomposition-of-a-matrix-in-the-pauli-basis
6. https://en.wikipedia.org/wiki/Ansatz
7. https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/variational_quantum_eigensolver.ipynb
8. https://michaelgoerz.net/notes/decomposing-two-qubit-hamiltonians-into-pauli-matrices.html
9. [Introduction to Quantum Computing and Quantum Hardware - Lecture 25 to 27](https://qiskit.org/learn/intro-qc-qh/)
10. Sim, Sukin, Peter D. Johnson, and Alán Aspuru‐Guzik. “Expressibility and Entangling Capability of Parameterized Quantum Circuits for Hybrid Quantum‐Classical Algorithms.” Advanced Quantum Technologies 2.12 (2019): 1900070. Crossref. Web.

## Additional Resources used for studying Variational Quantum Eigensolver
    
1. [Framework agnostic VQE tutorial by Alexander Soare](https://github.com/alexander-soare/framework-agnostic-vqe-tutorial)